In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.1'
#spark_version = 'spark-3.'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (100 kB/s)
Reading package lis

In [ ]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-17 18:22:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.72MB/s    in 0.2s    

2022-11-17 18:22:58 (5.72 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [85]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [86]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...|2015-08-31 00:00:00|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed| 

In [87]:
print(df.count())
df =df.dropna();
print(df.count())

5069140
5068421


In [88]:
print(df.count())
df=df.drop_duplicates()
print(df.count())

5068421
5068421


In [89]:
#examine the schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [91]:
#change format of review_date from string to date
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
df =df.withColumn('review_date',col('review_date').cast(DateType()))

In [92]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Create Table Dataframes

In [93]:
review_id_df= df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001KXUA9YP4X|   46128922|B00005JNWM|      93901757| 2005-06-08|
|R100DVELLPCTRN|   47061924|B002JUFPUE|     954468656| 2013-01-12|
|R100G4QMGXGC0Q|   49837360|B00HTLDULE|     540427023| 2014-11-04|
|R100L6085AB0OB|   35420271|B000212HCQ|     803561421| 2012-06-25|
|R100NK6RFSMOIH|   52258312|B00004UD5U|     853619966| 2009-07-01|
|R100PE4CFEKB3U|   49203858|B00009YXAW|     747270715| 2013-04-17|
|R100WNGCZ1YI5H|   38185846|B00005YU1M|     439322915| 2004-04-18|
|R100YEPR6VL1YB|   44403108|B0041KKYDI|     374170167| 2013-07-08|
|R10142BBJEM8OG|   39317350|B0002B55BG|      55334409| 2013-09-02|
|R10157LR18H0J8|   51217365|B00AZMFJYA|     645970389| 2013-12-20|
|R101BMFPF9JP5J|   49918685|B0001MZ7J6|     343439830| 2011-04-02|
|R101BYQJ30E5BF|   49862044|B00388PK1A|     875649142| 2013-01

In [94]:
products_df = df.select(['product_id','product_title'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00005JNWM|               Crash|
|B002JUFPUE|Dr. Seuss' How th...|
|B00HTLDULE|CASINO ROYALE/QUA...|
|B000212HCQ|Park BasiX: Zion ...|
|B00004UD5U|                Flip|
|B00009YXAW|The Brave Little ...|
|B00005YU1M|      Monster's Ball|
|B0041KKYDI|           127 Hours|
|B0002B55BG|         The Passion|
|B00AZMFJYA|Star Trek Into Da...|
|B0001MZ7J6|Jonny Quest - The...|
|B00388PK1A|The Wizard of Oz ...|
|B00000JGHT|           Deathtrap|
|B0046ZE3OE|Discover Tai Chi ...|
|B005OT810I|          Nightbreed|
|B00008H2GS|The White Sheik (...|
|B0013LL2Z2|         Logan's Run|
|B000KWZ1RU|     Soup of the Day|
|B000QGE8IS|             Othello|
|B000OPPBEQ|The Ultimate Matr...|
+----------+--------------------+
only showing top 20 rows



In [95]:
print(products_df.count())
products_df=products_df.drop_duplicates()
print(products_df.count())

5068421
297902


In [96]:
df.select(['customer_id']).distinct().count()

2075812

In [97]:
customers_df=df.groupby('customer_id').agg(count('customer_id').alias('customer_count'))

In [98]:
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35420271|             4|
|   29631384|             2|
|   11406322|             1|
|   16431227|             1|
|   24283512|            21|
|   20449184|            16|
|   52925615|            13|
|   27217320|            10|
|   50221281|             1|
|   13713446|            74|
|   12205680|             1|
|    3863609|             1|
|   51813840|             1|
|   12616774|             2|
|   25169471|             9|
|   47379040|             7|
|   50149916|             1|
|   49414964|             1|
|   13013055|             2|
|   20880642|             9|
+-----------+--------------+
only showing top 20 rows



In [99]:
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1001KXUA9YP4X|          4|            0|          2|   N|
|R100DVELLPCTRN|          5|            0|          0|   N|
|R100G4QMGXGC0Q|          5|            1|          1|   N|
|R100L6085AB0OB|          5|            1|          1|   N|
|R100NK6RFSMOIH|          2|            5|          7|   N|
|R100PE4CFEKB3U|          5|            0|          0|   N|
|R100WNGCZ1YI5H|          1|            6|         16|   N|
|R100YEPR6VL1YB|          4|            0|          0|   N|
|R10142BBJEM8OG|          5|            0|          0|   N|
|R10157LR18H0J8|          5|            1|          4|   N|
|R101BMFPF9JP5J|          5|            0|          0|   N|
|R101BYQJ30E5BF|          5|            0|          1|   N|
|R101CVWPI4RYC7|          5|            1|          2|   N|
|R101FLFZUI37D3|          5|            

In [100]:
# Load Dataframes into Postgres SQL DB


In [102]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://:5432/postgres"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [103]:
# Write DataFrame to table created above

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [104]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [105]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [106]:
# Write DataFrame to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)